In [142]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.preprocessing as sp
from sklearn.cluster import KMeans
%matplotlib inline
sns.set_style('whitegrid')
sns.set_context("poster")
sns.set_context("talk", 1.0, {"lines.linewidth": 3})

In [143]:
# パターン13の各受信機データを取得
df13_1 = pd.read_csv("./取得データセット/受信機1/rasp1_pattern13.csv")
df13_2 = pd.read_csv("./取得データセット/受信機2/nkmr2-dataset13.csv")
df13_3 = pd.read_csv("./取得データセット/受信機3/rasp3_pattern13.csv")
df13_4 = pd.read_csv("./取得データセット/受信機4/nkmr4-dataset13.csv")
df13_2 = df13_2.drop("\\N",axis=1)

#それぞれのカラム名の指定
df13_1.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp"]
df13_2.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp"]
df13_3.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp"]
df13_4.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp"]

#7つのビーコンのUUIDs
id1 = "10000000aaaa41441111110000000000"
id2 = "20000000aaaa41441111110000000000"
id3 = "30000000aaaa41441111110000000000"
id4 = "40000000aaaa41441111110000000000"
id5 = "50000000aaaa41441111110000000000"
id6 = "60000000aaaa41441111110000000000"
id7 = "70000000aaaa41441111110000000000"

### 元のデータに時刻のラベル付けをする関数

In [144]:
def add1(x):
    df13a = x[x.UUIDs == "1000000007ae1001b000001c4d8dffff"].drop_duplicates(['Minor'],keep = "last")
    df13b = x[x.UUIDs != "1000000007ae1001b000001c4d8dffff"]
    df13b = df13b.dropna()
    x = df13a.append(df13b)
    x = x.sort_index(ascending=True)
    
    id0 = "1000000007ae1001b000001c4d8dffff"
    x["time"] = x.Minor.where(x.UUIDs == id0,x[x.UUIDs == id0].Minor)
    x = x.fillna(method='ffill')
    x = x.fillna(1)
    x["time"] = x["time"].astype(int)
    return x
    #df13_2.time = df13_2.Minor.where(df13_2.UUIDs == "1000000007ae1001b000001c4d8dffff")

In [145]:
df13_1 = add1(df13_1)
df13_2 = add1(df13_2)
df13_3 = add1(df13_3)
df13_4 = add1(df13_4)

### 時刻のラベルを2秒ごとにする関数

In [146]:
def half(x):
    id0 = "1000000007ae1001b000001c4d8dffff"
    x["time"] = x.Minor.where(x.UUIDs == id0)
    x["time"] = (x["time"]-1).where(x.Minor % 2 == 0)
    x = x.fillna(method='ffill')
    x = x.fillna(1)
    x["time"] = x["time"].astype(int)

### 各送信機の時刻ごとの平均をとる関数

In [147]:
def add2(x,y,z):    
    df13c = []
    for n in pd.unique(x.time):
        a = x[(x.time == n) & (x.UUIDs == y)].RSSI.mean()
        a = round(a,1)
        df13c.append(a)


    df13s = pd.DataFrame(index=[],columns=[])
    df13s = df13s.append(df13c)
    df13s.columns = [z]
    df13s.index = pd.unique(x.time)
    x = df13s
    
    return x

In [148]:
df13_1s1 = add2(df13_1,id1,"nkmr-1")
df13_2s1 = add2(df13_2,id1,"nkmr-2")
df13_3s1 = add2(df13_3,id1,"nkmr-3")
df13_4s1 = add2(df13_4,id1,"nkmr-4")
df13s1 = pd.concat([df13_1s1,df13_2s1,df13_3s1,df13_4s1],axis=1)
df13s1 = df13s1.fillna(method = "ffill")
df13s1 = df13s1.fillna(-80)
df13s1["UUIDs"] = id1

In [149]:
df13_1s2 = add2(df13_1,id2,"nkmr-1")
df13_2s2 = add2(df13_2,id2,"nkmr-2")
df13_3s2 = add2(df13_3,id2,"nkmr-3")
df13_4s2 = add2(df13_4,id2,"nkmr-4")
df13s2 = pd.concat([df13_1s2,df13_2s2,df13_3s2,df13_4s2],axis=1)
df13s2 = df13s2.fillna(method = "ffill")
df13s2 = df13s2.fillna(-80)
df13s2["UUIDs"] = id2

In [150]:
df13_1s3 = add2(df13_1,id3,"nkmr-1")
df13_2s3 = add2(df13_2,id3,"nkmr-2")
df13_3s3 = add2(df13_3,id3,"nkmr-3")
df13_4s3 = add2(df13_4,id3,"nkmr-4")
df13s3 = pd.concat([df13_1s3,df13_2s3,df13_3s3,df13_4s3],axis=1)
df13s3 = df13s3.fillna(method = "ffill")
df13s3 = df13s3.fillna(-80)
df13s3["UUIDs"] = id3

In [151]:
df13_1s4 = add2(df13_1,id4,"nkmr-1")
df13_2s4 = add2(df13_2,id4,"nkmr-2")
df13_3s4 = add2(df13_3,id4,"nkmr-3")
df13_4s4 = add2(df13_4,id4,"nkmr-4")
df13s4 = pd.concat([df13_1s4,df13_2s4,df13_3s4,df13_4s4],axis=1)
df13s4 = df13s4.fillna(method = "ffill")
df13s4 = df13s4.fillna(-80)
df13s4["UUIDs"] = id4

In [152]:
df13_1s5 = add2(df13_1,id5,"nkmr-1")
df13_2s5 = add2(df13_2,id5,"nkmr-2")
df13_3s5 = add2(df13_3,id5,"nkmr-3")
df13_4s5 = add2(df13_4,id5,"nkmr-4")
df13s5 = pd.concat([df13_1s5,df13_2s5,df13_3s5,df13_4s5],axis=1)
df13s5 = df13s5.fillna(method = "ffill")
df13s5 = df13s5.fillna(-80)
df13s5["UUIDs"] = id5

In [153]:
df13_1s6 = add2(df13_1,id6,"nkmr-1")
df13_2s6 = add2(df13_2,id6,"nkmr-2")
df13_3s6 = add2(df13_3,id6,"nkmr-3")
df13_4s6 = add2(df13_4,id6,"nkmr-4")
df13s6 = pd.concat([df13_1s6,df13_2s6,df13_3s6,df13_4s6],axis=1)
df13s6 = df13s6.fillna(method = "ffill")
df13s6 = df13s6.fillna(-80)
df13s6["UUIDs"] = id6

In [154]:
df13_1s7 = add2(df13_1,id7,"nkmr-1")
df13_2s7 = add2(df13_2,id7,"nkmr-2")
df13_3s7 = add2(df13_3,id7,"nkmr-3")
df13_4s7 = add2(df13_4,id7,"nkmr-4")
df13s7 = pd.concat([df13_1s7,df13_2s7,df13_3s7,df13_4s7],axis=1)
df13s7 = df13s7.fillna(method = "ffill")
df13s7 = df13s7.fillna(-80)
df13s7["UUIDs"] = id7

### 元のデータにラベル付けしたデータ :
受信機1 : df13_1  
受信機2 : df13_2  
受信機3 : df13_3  
受信機4 : df13_4　　　　　　　

### 各送信機の時刻ごとに平均をとったデータ
ビーコン1 : df13s1  
ビーコン2 : df13s2  
ビーコン3 : df13s3  
ビーコン4 : df13s4  
ビーコン5 : df13s5  
ビーコン6 : df13s6  
ビーコン7 : df13s7  

In [155]:
#df13s1["date"] = df13_1[df13_1.UUIDs == id1].timestamp

In [166]:
df13s = df13s1.append(df13s2).append(df13s3).append(df13s4).append(df13s5).append(df13s6).append(df13s7).sort_index(ascending=True)
result1 = pd.DataFrame()
df13s

,nkmr-1,nkmr-2,nkmr-3,nkmr-4,UUIDs
1,-84.0,-80.0,-89.7,-88.7,10000000aaaa41441111110000000000
1,-89.4,-86.5,-90.3,-84.8,20000000aaaa41441111110000000000
1,-91.5,-84.0,-87.3,-83.0,70000000aaaa41441111110000000000
1,-80.5,-80.0,-80.1,-92.0,30000000aaaa41441111110000000000
1,-82.9,-80.0,-80.7,-89.2,60000000aaaa41441111110000000000
1,-84.5,-80.0,-82.6,-87.7,40000000aaaa41441111110000000000
1,-90.3,-76.0,-87.0,-80.4,50000000aaaa41441111110000000000
2,-89.0,-84.0,-86.5,-83.0,70000000aaaa41441111110000000000
2,-81.0,-80.0,-83.9,-87.7,40000000aaaa41441111110000000000
2,-89.5,-76.0,-85.7,-80.4,50000000aaaa41441111110000000000


### svmを行う関数

In [163]:
#svmを行う関数
def svm_act(data,id0):
    
    #学習用データの作成
    data["answer"] = data.UUIDs.apply(lambda x: 0 if x != id0 else 1)
    data = data.drop("UUIDs",axis=1)
    train_x = data[0:len(data) // 4 * 3]
    train_y = data[0:len(data) // 4 * 3][["answer"]]
    
    #テストデータの作成
    test_x = data[len(data)//4*3+1:len(data)]
    test_y = data[len(data)//4*3+1:len(data)][["answer"]]
    
    # ガウシアンカーネルによる SVM インスタンス生成
    clf = svm.SVC(kernel='rbf', C=1)

    # 教師データで学習 (データに対するフィッティング)
    clf.fit(train_x, train_y)

    # テストデータを分類する
    result = pd.Series(clf.predict(test_x))
    global result1
    result1 = pd.Series(clf.predict(test_x))
    
    #認識率を出力
    classification_report(test_y,result)
    result2 = accuracy_score(test_y,result)
    
    return result2

In [164]:
svm_act(df13s,id6)

/Users/tomanan/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.0

In [159]:
type(df13_1[0:35941][["RSSI"]])

pandas.core.frame.DataFrame

In [165]:
result1

0       1
1       0
2       0
3       0
4       0
5       1
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      1
17      0
18      0
19      0
20      0
21      0
22      0
23      1
24      0
25      0
26      0
27      0
28      1
29      0
       ..
2074    0
2075    1
2076    0
2077    0
2078    0
2079    0
2080    0
2081    1
2082    0
2083    0
2084    0
2085    1
2086    0
2087    0
2088    0
2089    0
2090    0
2091    1
2092    0
2093    0
2094    0
2095    0
2096    0
2097    1
2098    0
2099    0
2100    0
2101    0
2102    0
2103    0
Length: 2104, dtype: int64

In [ ]:
sns.countplot(df13_1[df13_1.UUIDs == id1].RSSI)
#sns.countplot(df13_1.RSSI)